## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [7]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "mlflow" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-54-227-73-187.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-54-227-73-187.compute-1.amazonaws.com:5000'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-will/0', creation_time=1689836785825, experiment_id='0', last_update_time=1689836785825, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

/home/william/.pyenv/versions/3.10.12/envs/zoomcamp-exp-track/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/william/.pyenv/versions/3.10.12/envs/zoomcamp-exp-track/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: 's3://mlflow-artifacts-remote-will/1/eeaaaa65a7ac46a19d653649387fc568/artifacts'


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-will/1', creation_time=1689838918139, experiment_id='1', last_update_time=1689838918139, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-will/0', creation_time=1689836785825, experiment_id='0', last_update_time=1689836785825, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [13]:
experiments = client.search_experiments()
for e in experiments:
    print(e.name, e.experiment_id)

my-experiment-1 1
Default 0


In [32]:
runs = client.search_runs(experiment_ids="1")
run_id = runs[0].info.run_id

In [33]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2023/07/20 16:01:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1689840063487, current_stage='None', description='', last_updated_timestamp=1689840063487, name='iris-classifier', run_id='eeaaaa65a7ac46a19d653649387fc568', run_link='', source='s3://mlflow-artifacts-remote-will/1/eeaaaa65a7ac46a19d653649387fc568/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>